# PS 88 Lab 11 - Causal Inference from Discontinuities

In [ ]:
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Part 1: Fake Discontinuities, Real Discontinuities

In order to be confident that our regression discontinuity methods can reliably inform us about causal effects, we want to know that they will detect jumps up or down in the outcome when they do exist, and *wont* detect jumps when they are not real. Like with other methods, we will first explore this with simulations where we know whether or not there is a causal effect of crossing a threshold, and then apply what we have learned to some real data.

First let's consider a hypothetical case of a "border discontinuity". Often times (particularly in colonial settings) borders between countries or other political units are drawn in a somewhat arbitray manner. 

Suppose a hypothetical country was historically divided between a northern portion which was colonized by the British and a southern portion colonized by the French. To simplify, let's assume the country lies on the equator, and the portion above the equator was run by the British while the portion below the equator was run by the French. Let's suppose this border was "arbitrary", in the sense that it didn't correspond to any pre-colonial border or post-colonial border. So, we can think of the main difference between regions close to the border is that some had French colonization while others had British colonization.

We might be interested in whether the different colonial legacies affect outcomes today. Let's consider the outcome of "average years of education", which could hypothetically be measured for a number of cities in contemporary times.

First we will simulate some data on the latitude of the city and educational outcomes.

To do so, we randomly generate each cities latitude, and then make a array corresponding to whether a city had British colonial rule (latitude above 0 means being above the equator). Next we simulate the educational attainment. Note this is a function of latitude (maybe for other historical reasons) but *not* a function of the `british` variable. Then we put everything in a data frame.

In [ ]:
lat = np.random.normal(0,1,size=500)
british = 1*(lat>0)
educ  = 10 + .3*lat + .6*np.sin(2*lat) + np.random.normal(0,.5, size=500)
dat = pd.DataFrame(data={"lat":lat, "educ":educ,"british":british})

Here is the relationship between latitude and educational attainment, also plotting cities that were formerly part of the French colony in blue and those from the British colony in orange.

In [ ]:
sns.scatterplot(x='lat',y='educ',data=dat, marker=".", hue="british")

By the way we cooked up the data, it appears there is much more educational attainment in the cities with British colonial rule. However, there is also a general trend where there is more education in general in cities that are further north (higher latitude). In fact, by the way we simulated the data, we know that there is no effect of British colonialism vs French colonialism. 

However, if we just took a difference of means we might think the British had a sizable effect on later educational attainment:

In [ ]:
np.mean(dat.loc[dat['british']==1, 'educ']), np.mean(dat.loc[dat['british']==0, 'educ']) 

I.e., the average for cities that were under British rule is 10.5, more than a year above the "ruled-by-French" average.

Now let's just compare places which are close to the pre-colonial border. To do this, we first define a "width" that we will use to decide what counts as close. First let's use a half-degree of latitude (about 35 miles). 

We first create arrays which indicate whether a city is between -.5 latitude and the equator (`close_under`) and between the equator and .5 latitude (`close_over`). Then we use the `.loc` function to identify educational outcomes for cities in these two groups, and take the means:

In [ ]:
width = .5
close_under = 1*(dat['lat'] > -width)*(dat['lat'] < 0)
close_over = 1*(dat['lat'] < width)*(dat['lat'] > 0)
np.mean(dat.loc[close_over==1, 'educ']), np.mean(dat.loc[close_under==1, 'educ']) 

**Question 1.1. Repeat this analysis with a width of .2. How does restricting attention to cities close to the equator affect the differences in educational attainment between cities that were under British vs. French rule?**

*Words for 1.1*

Now let's explore the effect of British rule using regression methods. First let's create separate data frames for the cities below and above the equator, and use `sns.regplot` to create best fit lines within these two groups.

To make the best fit line easier to see, we use the `marker="."` argument to make the point smaller and `scatter_kws={'alpha':0.3}` to make the scatterplot partially transparent.

In [ ]:
dat_under = dat[dat['lat'] < 0]
dat_over = dat[dat['lat'] >= 0]
sns.regplot(x="lat", y="educ", data=dat_under, marker=".", ci=None, scatter_kws={'alpha':0.3})
sns.regplot(x="lat", y="educ", data=dat_over, marker=".", ci=None, scatter_kws={'alpha':0.3})

From this it looks like there is a big jump up as we go from French rule (blue line) to British Rule (orange line). But we know there is no real causal effect here! The problem is that there is a nonlinear relationship between latitude and educational attainment, and so the best fit line underestimates the educational attainment for French colonies close to the border and overestimates educational attainment for British colonies close to the border. Combining, we get the appearance of British rule increasing educational attainment!

One thing that might help is to use best fit polynomials on both sides.

**Question 1.2. Add an "order=" argument to the `sns_regplot` commands to create the best fit quadratic on both sides of the equator. Then do the same for a best fit cubic (`order = 3`). How does using more flexible polynomials affect the "jump" at the border between French and British Rule?**

*Words for 1.2*

To do the comparison with linear trend lines more formally, we can run a regression with `smf.ols` that predicts educational attainment with latitude, whether it was under British rule, and the interaction term between the two.

In [ ]:
smf.ols('educ ~ lat + british + lat:british', data=dat).fit().summary()

**Question 1.3. Which of these coefficients corresponds to the "jump" from our `sns.regplot` with linear best fit lines on both sides of the equator? Interpret this coefficient.**

To fit more general polynomials on both sides of the equator, let's create some variables equal to the latitude squared and latitude cubed.

In [ ]:
dat['lat2'] = dat['lat']**2
dat['lat3'] = dat['lat']**3

**Question 1.4. Run the regression version of the regression plot with best fit quadratics on both sides (i.e., `order=2`). Hint: start with the previous regression, and add the latitude squared and the interaction between latitude squared and British rule.**

**Question 1.5 [Optional] Now run a regression which fits a best fit cubic equation on both sides of the equator. How does this change the coefficient on "British"?**

Now let's simulate a separate hypothetical country where there is a causal effect of British rule on education; let's say it increases it by a year (relative to French rule). Here is some simulated data for this. Again let's assume the equator is the dividing line between the British and French rule. 

In [ ]:
lat2 = np.random.normal(0,1,size=500)
british2 = 1*(lat2>0)
educ2  = 8 + -.5*lat2 + british2 + np.random.normal(0,1, size=500)
dat2 = pd.DataFrame(data={"lat":lat2, "educ":educ2,"british":british2})

Here is a scatterplot of the general trend: note it decreases slightly (given the negative coefficient when we generated `educ2`). 

In [ ]:
sns.scatterplot(x='lat', y='educ', data=dat2, ci=None, hue="british")

**1.6. Using at least one of the methods above (comparing cities just above and below the equator, running a regression with different linear/polynomial best fit lines above and below the equator), see if you can come up with an estimate for the causal effect of British vs. French rule on educational outcomes in country 2 which is close to the truth (an increase of one year). Hint: all of the code above should work, just change `dat` to `dat2`!**

# Part 2: Party Loyalty, Revisited. 

When we studied party competition, we looked at the relationship between voter preferences and politician behavior, asking whether this relationship reflected politicians catering to voters, or voters having a limited choice between electing someone from a party close to their preferences. 

We will revisit this topic and get a more precise answer to the question of whether there is a "party discipline" effect by looking at the difference in politician behavior in close elections. In particular, we will look at members of Congress in the US, and compare the voting behavior of representatives of districts that just barely elected a Democrat and those who barely elected a Republican. In such districts, there will not be a big difference in voter preferences, so if we see a "jump" in the behavior we can attribute that difference to a "party discipline" effect.

Here is some data from a <a href="http://public.econ.duke.edu/~psarcidi/lunchf08/leemorebutler.pdf">famous paper</a> by Lee, Moretti (of Berkeley!) and Butler. 

Each row corresponds to a member of Congress/term in office. 

In [ ]:
lmb = pd.read_csv("lmb.csv")
lmb

To make this more manageable, let's cut this down to a few key variables:
- `lagdemvoteshare` is the vote share of the democratic candidate in the previous election. So, when this is greater than .5, the democrat won
- `demvoteshare` is the democratic vote share in the next election
- `ada_vs` is a score of how often the candidate voted in line with the Americans for Democratic Action in the current term in office. Higher values mean more liberal voting. We'll refer to this as the "ADA score".
- `aclu_vs` is a similar score but done by the ACLU, which is more focused on civil liberties.

In [ ]:
lmb = lmb[["lagdemvoteshare", "ada_vs", "demvoteshare", "aclu_vs"]]
lmb

As a side note, there are lots of NaN's here, meaning we don't have data for that variable. By default the functions we use will drop these observations when making graphs/running regressions.

Let's first take a look at the bivariate relationship between how well the Democrat did in the election leading up to this term in office and the winning candidate ADA score.

In [ ]:
sns.regplot(x='lagdemvoteshare', y='ada_vs', data=lmb, marker='.', scatter_kws={'alpha':.2})

There is a pretty strong postive relationship here. Another thing to note is that there are a fair number of districts where the democratic vote share was exactly 0 or 1. These correspond to uncontested elections, where it is hard to say what the preferences of citizens are since they had only one choice. So let's drop those:

In [ ]:
lmb_comp = lmb[(lmb['lagdemvoteshare'] > 0) & (lmb['lagdemvoteshare'] < 1)]

Now let's redo the previous plot:

In [ ]:
sns.regplot(x='lagdemvoteshare', y='ada_vs', data=lmb_comp, marker='.', scatter_kws={'alpha':.2})

In addition to getting ride of the vertical lines of dots, this also leads to a steeper best fit line, since we are only focusing on cases where we have a reasonable measure of citizen preferences. 

If parties have a "disciplining" effect on members of congress, we would expect to see a "jump" in how often the member of congress votes in a liberal fashion. One way we can visualize whether such a sump happens is to create "bins" of lagged democratic vote share and plot the average ADA score for each bin.

To do this, we first create bins which effectively "round" the vote share to nearest percent. To do this, we first multiply the vote share by 100, and then use the "floor" function to drop the decimal. E.g., a vote share of .678 first gets turned in to 67.8, and then rounded down to 67, then convered back to range between 0 and 1 at .67.

In [ ]:
lmb['dembin'] = np.floor(100*lmb['lagdemvoteshare'])/100

We can then use the `.groupby()` function to compute the mean for each bin, and store the result in a data frame called `lmb_g`.

In [ ]:
lmb_g = lmb.groupby('dembin', as_index=False)['ada_vs'].mean()
lmb_g

Now let's plot the average of each bin:

In [ ]:
sns.scatterplot(x="dembin", y="ada_vs", data=lmb_g)
plt.axvline(.5)

It looks like there is a pretty big jump up in voting behavior as we go from electing a D to electing an R.

Now let's compare the average voting behavior of "barely elected Ds" vs "barely elected Rs". In particular, let's first define close elections as ones with a vote margin of less than 3%. So places with a vote share between 47% and 50% are "close R" districts and those between 50% and 53% are "close D" districts. We define vectors that indicate a close D and close R win, and then take the average to see how common these results are.

In [ ]:
width = .03
closeD = 1*(lmb_comp['lagdemvoteshare'] > .5)*(lmb_comp['lagdemvoteshare'] < .5 + width)
closeR = 1*(lmb_comp['lagdemvoteshare'] < .5)*(lmb_comp['lagdemvoteshare'] > .5 - width)
np.mean(closeD), np.mean(closeR)

This means that about 6% of elections are close R wins and 6% are close D wins. 

Now compare the average ADA score among these groups.

In [ ]:
np.mean(lmb_comp.loc[closeD==1, 'ada_vs']), np.mean(lmb_comp.loc[closeR==1, 'ada_vs'])

In [ ]:
np.mean(lmb_comp.loc[closeD==1, 'ada_vs'])- np.mean(lmb_comp.loc[closeR==1, 'ada_vs'])

So closely elected Ds' vote with the ADA over 20% more than closely elected Rs.

We can also look at the relationship between the democratic vote share before each session of congress and after.

In [ ]:
sns.regplot(x='lagdemvoteshare', y='demvoteshare', data=lmb_comp, marker='.', scatter_kws={'alpha':.2})

Not surprisingly, there is a strong positive relationship: if the Democrat did well in the previous election, they will probably do well in the next election, both due to voter preferences being pretty stable and some candidates being "higher quality" than others.

Another potential source of advantage is that if the Democrat won in the last election, they may have an easier tiem getting reelected for reasons like voters being more familiar with them. To see if such an effect exists (independent of the fact that Democratic vote share is positively correlated across elections, we can use the same "close election" methods.

**Question 2.1. Compute the difference in the democratic vote share in the *next* election (`demvoteshare`) among closely elected Democrats and Republicans. You can use the `closeD` and `closeR` we defined above to identify the closely elected Ds and Rs.**

**Question 2.2. This difference is often used to measure something called the *incumbency advantage*, because it tells us whether candidates who win close elections (and hence tend to be in close districts) tend to do better when they are running for re-election having already served in Congress (and hence are "incumbents"). What does your answer to question 2.1 tell us about the incumency advantage in these elections?**

*Words for 2.2*

Of course, both of these results may be sensitive to how we defined a "close" election. One way to explore how that definition affects the analysis is to just try a bunch of different values of "width" and see how that affects the difference of means we get. The following code does this for `width` ranging from 1% to 20%, using the ADA scores as the outcome.

In [ ]:
ws = np.arange(.01, .2,  .001)
est_ada = []
for width in ws:
    closeD = 1*(lmb_comp['lagdemvoteshare'] > .5)*(lmb_comp['lagdemvoteshare'] < .5 + width)
    closeR = 1*(lmb_comp['lagdemvoteshare'] < .5)*(lmb_comp['lagdemvoteshare'] > .5 - width)
    est_ada = np.append(est_ada, np.mean(lmb_comp.loc[closeD==1, 'ada_vs'])-  np.mean(lmb_comp.loc[closeR==1, 'ada_vs']))

In [ ]:
plt.plot(ws, est_ada)

If we define the width to be large, we end up comparing districts with reasonably different vote preferences. E.g., with the width of 20 we are comparing 50-70% D districts to 30-50% D districts, which are probably quite different. The drawback as we make the width small is that the estimate gets more volatile since there aren't a ton of members to compare. Still, for small width (e.g., below 5%) the estimate is generally between 15% and 20%, which isn't too crazy of a range. 

**Question 2.2. Create a similar graph using the next election D vote share. What do we learn from this graph?**

Now let's estimate this with some regressions. First, we create a variable equal to 1 if the democrat won, and separate data sets among the D winners and R winners

In [ ]:
lmb_comp['lagdemwin'] = 1*(lmb_comp['lagdemvoteshare'] > .5)
lmb_compR = lmb_comp[lmb_comp['lagdemwin']==0]
lmb_compD = lmb_comp[lmb_comp['lagdemwin']==1]

**Question 2.3. Use `sns.regplot` to plot a linear best fit line for the ADA score of districts where the R won, and then another `sns.regplot` to add the similar graph for the D's (refer back to the code between 1.1 and 1.2 for an example). Interpret this graph.**

To make the regression form easier, it will be helpful to "normalize" our running varable (here, `lagdemvoteshare`) in a way that the potential discontinuity happens at 0. To do that, let's create a variable called `lagdemdiff`.

In [ ]:
lmb_comp['lagdemdiff'] = lmb_comp['lagdemvoteshare'] - .5

In [ ]:
rd1 = smf.ols('ada_vs ~ lagdemdiff + lagdemwin + lagdemwin:lagdemdiff', data=lmb_comp).fit()
rd1.summary()

**Question 2.4. Intepret the results of this regression. How does it compare to the estimated effect of party influence from comparing close elections?**

**Question 2.5. Run a similiar regression to estimate the incumbency advantage, i.e., how much better the Democrat does in the next election if they win the previous election and so get to run as an incumbent.**

**Question 2.6 [Optional]. See how your answer to 2.4 and 2.5 changes if you use best fit polynomials rather than a best fit line for each group.**